# First steps

In [ ]:
import sisl
# We define the root directory where our files are
siesta_files = sisl._environ.get_environ_variable("SISL_FILES_TESTS") / "sisl" / "io" / "siesta"

## Activating the viz framework

The first thing you will need to do in order to use plots is to import `sisl.viz`:

In [ ]:
import sisl.viz

This will load the appropiate things into sisl to use the visualization tools. You can also control the loading of the framework with an environment variable:

```
SISL_VIZ_AUTOLOAD=True
```

will load the framework on `import sisl`, so you won't need to explicitly import it.

<div class="alert alert-info">
    
Note
    
If you use sisl to run high performance calculations where you initialize sisl frequently it's better to have the autoloading turned off (default), as it might introduce an overhead.
    
</div>

Now that the framework has been loaded, we can start plotting!

## Your first plots

The most straightforward way to plot things in sisl is to call their `plot` method. For example if we have the path to a bands file we can call plot:

In [ ]:
sisl.get_sile(siesta_files / "SrTiO3.bands").plot()

You can pass arguments to the plotting function:

In [ ]:
rho_file = sisl.get_sile(siesta_files / "SrTiO3.RHO")
rho_file.plot(axes="xy", nsc=[2,1,1], smooth=True)

Some objects can be plotted in different ways, and just calling `plot` will do it in the default way. You can however **choose which plot you want** from the available representations. For example, out of a PDOS file you can plot the PDOS (the default):

In [ ]:
pdos_file = sisl.get_sile(siesta_files / "SrTiO3.PDOS")

pdos_file.plot(groups=[{"species": "O", "name": "O"}, {"species": "Ti", "name": "Ti"}])

or the geometry (not the default, you need to specify it):

In [ ]:
pdos_file.plot.geometry(atoms_scale=0.7)

Updating your plots
----------------

When you call `.plot()`, you receive a `Plot` object:

In [ ]:
pdos_plot = pdos_file.plot()
type(pdos_plot)

`Plot` objects are a kind of `Workflow`. You can check the `sisl.nodes` documentation to understand what exactly this means. But long story short, this means that the computation is split in multiple nodes, as you can see in the following diagram:


In [ ]:
pdos_plot.network.visualize(notebook=True)

With that knowledge, when you update the inputs of a plot, only the necessary parts are recalculated. In that way, you may avoid repeating expensive calculations or reading to files that no longer exist. Inputs are updated with `update_inputs`:

In [ ]:
pdos_plot.update_inputs(Erange=[-3, 3])

Some inputs are a bit cumbersome to write by hand, and therefore along your journey you'll find that plots have some helper methods to modify inputs much faster. For example, `PdosPlot` has the `split_DOS` method, which generates groups of orbitals for you.

In [ ]:
pdos_plot.split_DOS()

<div class="alert alert-info">
    
Don't worry!
    
Each plot class has its own dedicated notebook in the documentation to guide you through all the knobs that they have!
    
</div>



## Different plotting backends

Hidden between all the inputs, you can find a **very special input**: `backend`.

This input allows you to choose the plotting backend used to display a plot. If you don't like the default one, just change it!

In [ ]:
pdos_plot.update_inputs(backend="matplotlib")

Let's go back to the default one, `plotly`.

In [ ]:
pdos_plot.update_inputs(backend="plotly")

## Further customization

If you are a master of some backend, you'll be happy to know that you can run any backend specific method on the plot. For example, plotly has a method called `add_vline` that draws a vertical line:

In [ ]:
pdos_plot.add_vline(-1).add_vline(2)

In fact, if you need the raw figure for something, you can find it under the `figure` attribute.

In [ ]:
type(pdos_plot.figure)

## Discover more

This notebook has shown you the most basic features of the framework with the hope that you will be hooked into it :)

If it succeeded, we invite you to check the rest of the documentation. **It only gets better from here!**

_______________
This next cell is just to create the thumbnail for the notebook in the docs

In [ ]:
thumbnail_plot = rho_file.plot(axes="xy", nsc=[2,1,1], smooth=True)

if thumbnail_plot:
    thumbnail_plot.show("png")